In [4]:

# !pip install mysql-connector-python

# levantar la base de datos: mysql.server start
# bajar la base de datos: mysql.server stop
# reiniciar la base de datos: mysql.server restart

# ANTES DE INICIAR, SE CARGARON TODOS LOS EXCEL PASADOS A LA BASE DE DATOS.
# La data original fue cargada como tablas y desde SQL convertidos al nuevo
# modelo (tablas que inician con f_%)

# para exportar la base de datos
# mysqldump -u proyectodegrado2 -p proyectodegrado2 -h localhost > archivo.sql
# pide password: pg2

# para importar la base de datos
# mysql -u proyectodegrado2 -p proyectodegrado2 -h localhost < archivo.sql
# pide password: pg2
# la bsae de datos debe estar creada y el usuario uaopiloto2 tambien con password up2 y 
# permisos de sql todos, y de administrador le puse dba


import numpy as np
import pandas as pd
from sqlalchemy import create_engine, text
import mysql.connector   # es bueno instalar sqlalchemy y mysql.connector para facilitar luego DML y SELECT.

usuario = 'proyectodegrado2'
contrasena = 'pg2'
host = 'localhost'  # Por ejemplo, 'localhost'
base_de_datos = 'proyectodegrado2'

# Crea una cadena de conexión
cadena_conexion = f'mysql+mysqlconnector://{usuario}:{contrasena}@{host}/{base_de_datos}'

# Crea una instancia del motor SQLAlchemy
motor = create_engine(cadena_conexion)


Todas las transformaciones se hicieron en el archivo sql en **transformacion_datos.sql**


In [129]:
# ejemplo de cómo ejecutar un DML
drop_tabla = text(""" drop table temp""")

crea_tabla = text("""
    create table temp as select count(*) cuantos from f_estudiantes
""") 

# Ejecuta la instrucción SQL utilizando la conexión del motor
with motor.connect() as connection:
    try:
        connection.execute(drop_tabla)
        print('borró temp')
    except:
        pass # si la tabla no existe saca error porque no puede hacer el drop

    connection.execute(crea_tabla)
    print('creó temp')

borró temp
creó temp


## **0. Creamos la tabla base DATOS basada en F_MatriculasAsignaturas**

Para las filas con marca "analizar = 1", se indica cuales filas tienen que ver con matrículas de las asignaturas de interés.  en este caso:  300MAG006 y 300MAG007.

hay casos de filas sin docente, entonces, usamos el docente 'Ninguno', con edad promedio (43), y sexo mayority (M).

MEJORAR:
Se puede luego optimizar el cálculo de la edad promedio al momento de determinar cual asigantura está tomando el estudiante

In [130]:
# decido usar mysql-connect en vez de alchemy porque hacer un DML con alchemy es un dolor de huevo
query = """
    SELECT  ma.key_estudiante, ma.periodo, ma.asignatura, 
		TIMESTAMPDIFF(YEAR,e.birthdate,p.fecha_inicio) estudiante_edad, 
        e.sex estudiante_genero, IFNULL(e.cod_colegio,'Ninguno') colegio,
        IFNULL(ma.key_docente,'Ninguno') key_docente, IFNULL(d.sex,1) docente_genero,
        IFNULL(TIMESTAMPDIFF(YEAR,d.birthdate,p.fecha_inicio),43) docente_edad, 
        ma.estado, IFNULL(CAST(ma.nota AS DECIMAL(10,2)),-1) nota
    FROM f_matriculasAsignaturas ma
    INNER JOIN f_estudiantes e
    ON e.key_estudiante = ma.key_estudiante
    LEFT JOIN f_docentes d 
    ON d.key_docente = ma.key_docente
    INNER JOIN f_periodos p
    ON p.periodo = ma.periodo
    WHERE ma.analizar = 1;
"""

DATOS = pd.read_sql(query, con=motor)
DATOS.head()


,key_estudiante,periodo,asignatura,estudiante_edad,estudiante_genero,colegio,key_docente,docente_genero,docente_edad,estado,nota
0,306023AB9393647A8E4AD19E67EC6AB620240D78,20151,300MAG006,18,1,0047377,Ninguno,1,43,E,3.0
1,306023AB9393647A8E4AD19E67EC6AB620240D78,20152,300MAG007,19,1,0047377,Ninguno,1,43,E,1.5
2,1E8888869DC4F3ED9EC8F786AC7377C44CAAEA3D,20152,300MAG006,19,1,0203539,Ninguno,1,43,E,2.2
3,1BF76E980C707F5F18F9C3FC6EC18D828DC371CA,20152,300MAG006,18,1,0206317,Ninguno,1,43,E,1.7
4,1917B5D7ADFF357668C8A6F0A8EAF0638DF6B1E7,20152,300MAG006,17,1,0047257,Ninguno,1,43,E,1.8


In [156]:
DATOS.shape

(7145, 841)

pivoteaemos primero el **docente** con que tomó la clase para hacer hot_encoding

In [157]:
temp = DATOS[['key_estudiante','periodo','asignatura','key_docente']].copy()
temp['valor'] = 1
temp.head()

,key_estudiante,periodo,asignatura,key_docente,valor
0,306023AB9393647A8E4AD19E67EC6AB620240D78,20151,300MAG006,Ninguno,1
1,306023AB9393647A8E4AD19E67EC6AB620240D78,20152,300MAG007,Ninguno,1
2,1E8888869DC4F3ED9EC8F786AC7377C44CAAEA3D,20152,300MAG006,Ninguno,1
3,1BF76E980C707F5F18F9C3FC6EC18D828DC371CA,20152,300MAG006,Ninguno,1
4,1917B5D7ADFF357668C8A6F0A8EAF0638DF6B1E7,20152,300MAG006,Ninguno,1


In [158]:
temp_pivot = temp.pivot(index=['key_estudiante','periodo','asignatura'], columns='key_docente', values='valor').reset_index()
temp_pivot = temp_pivot.add_prefix('docente_')
temp_pivot.rename(columns={'docente_key_estudiante':'key_estudiante',
                           'docente_periodo':'periodo',
                           'docente_asignatura':'asignatura'}, 
                           inplace=True)
temp_pivot.head()

key_docente,key_estudiante,periodo,asignatura,docente_05D88C86A8703C7D6AA84290EF2E50B8A333F91E,docente_0B9C1D303F4FEC9D2044DBF9B694574AC8D19074,docente_152CB4A3B74E5F9ECD59D202E88577E4CABC335D,docente_296F3C7FA0ED301F8DB1E32B66D6394D9825676E,docente_2E1098D84E806AA82A86956FF4AA97E13D71A2FE,docente_329E7BBDF9F5017D09076B269FF1BBF695FDC18F,docente_32CE450FFE92C23D0DB5C2E1F370427AC62A5804,...,docente_7F11236122B24A2A4BD9BD618CC344974A8C599A,docente_8E5EE1BFD0FAFBA27E7B928C4AD33AAE1F926657,docente_AC970FD6C97087F3E27B190E51CE88B8E1B8C372,docente_B179BCCB45E42E068F79EAB481B74C769BB104DC,docente_BB9022363551E68494D8ABBC820AA8952CB3CF50,docente_D3EA163FAB6876C738502F4CF40005D58EA4C81F,docente_DEE1E3CE850B388929CC4355B8D884E9A61C1724,docente_E1D257E350638DC8460A0FE39B6356FFF4BFABD1,docente_EB6BEE0D72D709BD8120FBBBFD6096ADB5AC65B1,docente_Ninguno
0,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20181,300MAG006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20181,300MAG007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20182,300MAG006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20182,300MAG007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,00724CD562AF1C6FF4ED73991F1DA0BB174A0727,20182,300MAG006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Primero pivoteamos el **docente** para hacer hot-enconding

In [159]:
DATOS = DATOS.merge(temp_pivot, on=['key_estudiante','periodo','asignatura'], how='left')
DATOS = DATOS.fillna(0) # 0: no tomó la clase con ese docente.  1: si la tomó con ese docente

In [160]:
DATOS.head()

,key_estudiante,periodo,asignatura,estudiante_edad,estudiante_genero,colegio,key_docente,docente_genero,docente_edad,estado,...,docente_7F11236122B24A2A4BD9BD618CC344974A8C599A_y,docente_8E5EE1BFD0FAFBA27E7B928C4AD33AAE1F926657_y,docente_AC970FD6C97087F3E27B190E51CE88B8E1B8C372_y,docente_B179BCCB45E42E068F79EAB481B74C769BB104DC_y,docente_BB9022363551E68494D8ABBC820AA8952CB3CF50_y,docente_D3EA163FAB6876C738502F4CF40005D58EA4C81F_y,docente_DEE1E3CE850B388929CC4355B8D884E9A61C1724_y,docente_E1D257E350638DC8460A0FE39B6356FFF4BFABD1_y,docente_EB6BEE0D72D709BD8120FBBBFD6096ADB5AC65B1_y,docente_Ninguno_y
0,306023AB9393647A8E4AD19E67EC6AB620240D78,20151,300MAG006,18,1,0047377,Ninguno,1,43,E,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,306023AB9393647A8E4AD19E67EC6AB620240D78,20152,300MAG007,19,1,0047377,Ninguno,1,43,E,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,1E8888869DC4F3ED9EC8F786AC7377C44CAAEA3D,20152,300MAG006,19,1,0203539,Ninguno,1,43,E,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,1BF76E980C707F5F18F9C3FC6EC18D828DC371CA,20152,300MAG006,18,1,0206317,Ninguno,1,43,E,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,1917B5D7ADFF357668C8A6F0A8EAF0638DF6B1E7,20152,300MAG006,17,1,0047257,Ninguno,1,43,E,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


ahora pivoteamos el **colegio** para hacer hot-enconding

In [161]:
temp = DATOS[['key_estudiante','periodo','asignatura','colegio']].copy()
temp['valor'] = 1
temp.head()

,key_estudiante,periodo,asignatura,colegio,valor
0,306023AB9393647A8E4AD19E67EC6AB620240D78,20151,300MAG006,0047377,1
1,306023AB9393647A8E4AD19E67EC6AB620240D78,20152,300MAG007,0047377,1
2,1E8888869DC4F3ED9EC8F786AC7377C44CAAEA3D,20152,300MAG006,0203539,1
3,1BF76E980C707F5F18F9C3FC6EC18D828DC371CA,20152,300MAG006,0206317,1
4,1917B5D7ADFF357668C8A6F0A8EAF0638DF6B1E7,20152,300MAG006,0047257,1


In [162]:
temp_pivot = temp.pivot(index=['key_estudiante','periodo','asignatura'], columns='colegio', values='valor').reset_index()
prefijo = 'colegio_'
temp_pivot = temp_pivot.add_prefix(prefijo)
temp_pivot.rename(columns={prefijo + 'key_estudiante':'key_estudiante',
                           prefijo + 'periodo':'periodo',
                           prefijo + 'asignatura':'asignatura'}, 
                           inplace=True)
temp_pivot.head()

colegio,key_estudiante,periodo,asignatura,colegio_0040007,colegio_0040102,colegio_0040110,colegio_0040115,colegio_0040464,colegio_0040508,colegio_0040519,...,colegio_8957635,colegio_8959798,colegio_8966791,colegio_8966932,colegio_8967201,colegio_8969340,colegio_8971605,colegio_8982673,colegio_8985906,colegio_OTHR
0,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20181,300MAG006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20181,300MAG007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20182,300MAG006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20182,300MAG007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,00724CD562AF1C6FF4ED73991F1DA0BB174A0727,20182,300MAG006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [163]:
DATOS = DATOS.merge(temp_pivot, on=['key_estudiante','periodo','asignatura'], how='left')
DATOS = DATOS.fillna(0) # 0: no tomó la clase con ese docente.  1: si la tomó con ese docente
DATOS.head()

,key_estudiante,periodo,asignatura,estudiante_edad,estudiante_genero,colegio,key_docente,docente_genero,docente_edad,estado,...,colegio_8957635_y,colegio_8959798_y,colegio_8966791_y,colegio_8966932_y,colegio_8967201_y,colegio_8969340_y,colegio_8971605_y,colegio_8982673_y,colegio_8985906_y,colegio_OTHR_y
0,306023AB9393647A8E4AD19E67EC6AB620240D78,20151,300MAG006,18,1,0047377,Ninguno,1,43,E,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,306023AB9393647A8E4AD19E67EC6AB620240D78,20152,300MAG007,19,1,0047377,Ninguno,1,43,E,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1E8888869DC4F3ED9EC8F786AC7377C44CAAEA3D,20152,300MAG006,19,1,0203539,Ninguno,1,43,E,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1BF76E980C707F5F18F9C3FC6EC18D828DC371CA,20152,300MAG006,18,1,0206317,Ninguno,1,43,E,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1917B5D7ADFF357668C8A6F0A8EAF0638DF6B1E7,20152,300MAG006,17,1,0047257,Ninguno,1,43,E,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [164]:
DATOS.shape

(7145, 1524)

In [165]:
DATOS.to_pickle('checkPoints/DATOS0.pkl')

## **1. LABEL: calculamos si dada una fila en d de DATOS, si cancelo o perdió, o ganó**

dada una fila, miramos si está con estado D (DROP).  LABEL = 0.
Si está en estado E con nota < 30,     LABEL = 0
si esta en estado E con nota >= 3.0,   LABEL = 1
otro caso:  LABEL = -1

In [166]:
DATOS = pd.read_pickle('checkPoints/DATOS0.pkl')

In [167]:
def calcularLABEL(row):

    # Si el estado es cancelado (DROP) se retorna 1  (estos casos tienen nota nula)
    if row['estado'] == 'D':
        return 1 
    
    # si el estado es matriculado (ENROLL)
    if row['estado'] == 'E':

        # si la nota es nula (-1), es porque la está viendo y por tanto es una fila para TEST, no para TRAINING
        if row['nota']==-1:
            return -1
        # si la nota es menor a 3, la perdió y por tanto se marca con un 1
        if row['nota']<3.0:
            return 1
        else:
            # si no, no la perdió y por tanto se marca con un 0.
            return 0 

DATOS['LABEL'] = DATOS.apply(calcularLABEL, axis=1)
        

In [168]:
DATOS['LABEL'].value_counts()

LABEL
 0    4820
 1    1791
-1     534
Name: count, dtype: int64

Salvamos este punto como DATA2 como pickle

In [169]:
DATOS.to_pickle('checkPoints/DATOS1.pkl')

In [170]:
DATOS.shape

(7145, 1524)

## **2. ICFES**

obtenemos las áreas del icfes y las adicionamos a DATA.
la tabla original venía con el nombre del examen (test_id) y el nombre de los componentees (ej: BIO).  sin embargo, aparecen algunos que sugieren que son items (ej: BIO1).  Hay también totales (TOT).

Para no eliminar nada, se trata cada uno como si fuera un item distinto, concatenando el **test_id** con el **test_component**.

In [171]:
DATOS = pd.read_pickle('checkPoints/DATOS1.pkl')

# decido usar mysql-connect en vez de alchemy porque hacer un DML con alchemy es un dolor de huevo
query = """
    SELECT  key_estudiante, CONCAT(test_id,'_',test_component) item, score
    FROM f_icfes
    ORDER BY test_id, CONCAT(test_id,'_',test_component);
"""
temp = pd.read_sql(query, con=motor)
temp.head()

,key_estudiante,item,score
0,4C5221320BDF0B397D413BCD1BE8706973F85F3F,ICF_1_APM,48.0
1,4C5221320BDF0B397D413BCD1BE8706973F85F3F,ICF_1_APV,58.0
2,4C5221320BDF0B397D413BCD1BE8706973F85F3F,ICF_1_BIO,59.0
3,4C5221320BDF0B397D413BCD1BE8706973F85F3F,ICF_1_COM,51.0
4,4C5221320BDF0B397D413BCD1BE8706973F85F3F,ICF_1_ELE,55.0


In [172]:
temp_pivot = temp.pivot(index=['key_estudiante'], columns='item', values='score').reset_index()
prefijo = 'icfes_'
temp_pivot = temp_pivot.add_prefix(prefijo)
temp_pivot.rename(columns={prefijo + 'key_estudiante':'key_estudiante',
                           prefijo + 'periodo':'periodo',
                           prefijo + 'asignatura':'asignatura'}, 
                           inplace=True)
temp_pivot.head()

item,key_estudiante,icfes_ICFES3_CCIUD,icfes_ICFES3_DCC,icfes_ICFES3_DCN,icfes_ICFES3_DIN,icfes_ICFES3_DLC,icfes_ICFES3_DMA,icfes_ICFES3_DRC,icfes_ICFES3_DSC,icfes_ICFES3_IPEM,...,icfes_ICF_2_QUIC1,icfes_ICF_2_QUIC2,icfes_ICF_2_QUIC3,icfes_ICF_2_QUIG1,icfes_ICF_2_QUIG2,icfes_ICF_2_QUIG3,icfes_ICF_2_QUIG4,icfes_ICF_2_QUIPR,icfes_ICF_2_SOC,icfes_ICF_2_TOT
0,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,00724CD562AF1C6FF4ED73991F1DA0BB174A0727,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,007A5DF5947E08FEBA1AED5A442DE6199D907121,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,007AD6584DA00DEE96BA57A369CDAB8C9C832D53,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,00844F609DA54DFF214529B66202E11E4D020184,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [173]:
DATOS = DATOS.merge(temp_pivot, on=['key_estudiante'], how='left')
DATOS = DATOS.fillna(-100) # -100: el icfes de ese estudiante no tiene dicho componente
DATOS.head()

,key_estudiante,periodo,asignatura,estudiante_edad,estudiante_genero,colegio,key_docente,docente_genero,docente_edad,estado,...,icfes_ICF_2_QUIC1_y,icfes_ICF_2_QUIC2_y,icfes_ICF_2_QUIC3_y,icfes_ICF_2_QUIG1_y,icfes_ICF_2_QUIG2_y,icfes_ICF_2_QUIG3_y,icfes_ICF_2_QUIG4_y,icfes_ICF_2_QUIPR_y,icfes_ICF_2_SOC_y,icfes_ICF_2_TOT_y
0,306023AB9393647A8E4AD19E67EC6AB620240D78,20151,300MAG006,18,1,0047377,Ninguno,1,43,E,...,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,0.0
1,306023AB9393647A8E4AD19E67EC6AB620240D78,20152,300MAG007,19,1,0047377,Ninguno,1,43,E,...,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,0.0
2,1E8888869DC4F3ED9EC8F786AC7377C44CAAEA3D,20152,300MAG006,19,1,0203539,Ninguno,1,43,E,...,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0
3,1BF76E980C707F5F18F9C3FC6EC18D828DC371CA,20152,300MAG006,18,1,0206317,Ninguno,1,43,E,...,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0,-100.0
4,1917B5D7ADFF357668C8A6F0A8EAF0638DF6B1E7,20152,300MAG006,17,1,0047257,Ninguno,1,43,E,...,8.0,8.2,6.1,9.6,5.8,7.2,7.4,0.0,-100.0,348.0


In [174]:
DATOS.to_pickle('checkPoints/DATOS2.pkl')

In [175]:
DATOS.shape

(7145, 1647)

## **3. promedio nivel nota asignaturas en semestres anteriores**

Promedio nivel_nota obtenido por semestres anteriores.
Se calcula el promedio de nivel nota por semestre, se mira el orden de este y se calcula
la diferencia contra el orden del periodo actual


In [176]:
DATOS = pd.read_pickle('checkPoints/DATOS2.pkl')

# OJO:   Nivel nota tiene la siguiente informacion
# -10:   los estudiante que apenas están viendo la asignatura (periodo 20231)
# - 5:   los que la cancelaron
#   0    los que la perdieron
#   1:   los que la ganaron regularimbis
#   2:   los que la ganaron bien (igual o mayor a 3.8) 
query = """
    SELECT ma.key_estudiante, ma.periodo, ma.asignatura, 
           p.ordenRE - p2.ordenRE hace, AVG(ma2.nivel_nota) promedio
    FROM f_matriculasAsignaturas ma						-- tabla base
    INNER JOIN f_periodos p								--  periodo de ma
    ON p.periodo = ma.periodo
    INNER JOIN  f_matriculasAsignaturas ma2				-- las asignaturas que vio antes
    ON  ma2.key_estudiante = ma.key_estudiante
    INNER JOIN f_periodos p2							-- periodo de las asignaturas que vio antes
    ON p2.periodo = ma2.periodo
    WHERE ma.analizar = 1								-- se toman en cuenta solo analizar = 1
    AND p2.tipo = 'RE'									-- solo se miran asig anteriores vistas en períodos regulares
    AND p2.ordenRE < p.ordenRE                          -- solo miramos en el orden correcto
    GROUP BY ma.key_estudiante, ma.periodo, ma.asignatura, p.ordenRE - p2.ordenRE;
"""
temp = pd.read_sql(query, con=motor)
temp.head()

,key_estudiante,periodo,asignatura,hace,promedio
0,007AD6584DA00DEE96BA57A369CDAB8C9C832D53,20182,300MAG007,2.0,1.5714
1,007AD6584DA00DEE96BA57A369CDAB8C9C832D53,20182,300MAG007,1.0,2.0000
2,AA8834B56776F3D46C0C7983349F9515B1A686E7,20181,300MAG007,4.0,1.5000
3,AA8834B56776F3D46C0C7983349F9515B1A686E7,20181,300MAG007,3.0,0.0000
4,AA8834B56776F3D46C0C7983349F9515B1A686E7,20181,300MAG007,2.0,1.5714


In [177]:
temp_pivot = temp.pivot(index=['key_estudiante','periodo','asignatura'], columns='hace', values='promedio').reset_index()
prefijo = 'prom_nivel_hace_'
temp_pivot = temp_pivot.add_prefix(prefijo)
temp_pivot.rename(columns={prefijo + 'key_estudiante':'key_estudiante',
                           prefijo + 'periodo':'periodo',
                           prefijo + 'asignatura':'asignatura'}, 
                           inplace=True)
temp_pivot.head()

hace,key_estudiante,periodo,asignatura,prom_nivel_hace_1.0,prom_nivel_hace_2.0,prom_nivel_hace_3.0,prom_nivel_hace_4.0,prom_nivel_hace_5.0,prom_nivel_hace_6.0,prom_nivel_hace_7.0,...,prom_nivel_hace_14.0,prom_nivel_hace_15.0,prom_nivel_hace_16.0,prom_nivel_hace_17.0,prom_nivel_hace_18.0,prom_nivel_hace_19.0,prom_nivel_hace_21.0,prom_nivel_hace_22.0,prom_nivel_hace_23.0,prom_nivel_hace_24.0
0,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20181,300MAG006,2.0000,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20181,300MAG007,2.0000,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20182,300MAG006,2.0000,2.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20182,300MAG007,2.0000,2.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,00724CD562AF1C6FF4ED73991F1DA0BB174A0727,20182,300MAG006,-0.1429,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [178]:
temp_pivot.shape

(6932, 26)

In [180]:
DATOS = DATOS.merge(temp_pivot, on=['key_estudiante','periodo','asignatura'], how='left')
DATOS = DATOS.fillna(-10) # -10: si no es posible calcular un periodo tan anterior (no estaba el estud aun)
DATOS.head()

,key_estudiante,periodo,asignatura,estudiante_edad,estudiante_genero,colegio,key_docente,docente_genero,docente_edad,estado,...,prom_nivel_hace_14.0,prom_nivel_hace_15.0,prom_nivel_hace_16.0,prom_nivel_hace_17.0,prom_nivel_hace_18.0,prom_nivel_hace_19.0,prom_nivel_hace_21.0,prom_nivel_hace_22.0,prom_nivel_hace_23.0,prom_nivel_hace_24.0
0,306023AB9393647A8E4AD19E67EC6AB620240D78,20151,300MAG006,18,1,0047377,Ninguno,1,43,E,...,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0
1,306023AB9393647A8E4AD19E67EC6AB620240D78,20152,300MAG007,19,1,0047377,Ninguno,1,43,E,...,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0
2,1E8888869DC4F3ED9EC8F786AC7377C44CAAEA3D,20152,300MAG006,19,1,0203539,Ninguno,1,43,E,...,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0
3,1BF76E980C707F5F18F9C3FC6EC18D828DC371CA,20152,300MAG006,18,1,0206317,Ninguno,1,43,E,...,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0
4,1917B5D7ADFF357668C8A6F0A8EAF0638DF6B1E7,20152,300MAG006,17,1,0047257,Ninguno,1,43,E,...,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0


In [182]:
DATOS.to_pickle('checkPoints/DATOS3.pkl')
DATOS.shape


(7145, 1693)

## **4. promedio nivel nota asignaturas por departamento, de asignaturas anteriores**

Promedio nivel_nota obtenido por semestres anteriores.
Se calcula el promedio de nivel nota por dpto, se mira el orden de este y se calcula
la diferencia contra el orden del periodo actual


In [185]:
DATOS = pd.read_pickle('checkPoints/DATOS3.pkl')

# decido usar mysql-connect en vez de alchemy porque hacer un DML con alchemy es un dolor de huevo
query = """
    SELECT ma.key_estudiante, ma.periodo, ma.asignatura, 
           ma2.departamento, AVG(ma2.nivel_nota) promedio
    FROM f_matriculasAsignaturas ma						-- tabla base
    INNER JOIN f_periodos p								--  periodo de ma
    ON p.periodo = ma.periodo
    INNER JOIN  f_matriculasAsignaturas ma2				-- las asignaturas que vio antes
    ON  ma2.key_estudiante = ma.key_estudiante
    INNER JOIN f_periodos p2							-- periodo de las asignaturas que vio antes
    ON p2.periodo = ma2.periodo
    WHERE ma.analizar = 1								-- se toman en cuenta solo analizar = 1
    AND p2.orden < p.orden                          -- solo miramos en el orden correcto
    GROUP BY ma.key_estudiante, ma.periodo, ma.asignatura, ma2.departamento;
"""
temp = pd.read_sql(query, con=motor)
temp.head()

,key_estudiante,periodo,asignatura,departamento,promedio
0,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20181,300MAG006,CIP,2.0
1,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20181,300MAG006,CSP,2.0
2,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20181,300MAG006,EIH,2.0
3,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20181,300MAG006,EPG,2.0
4,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20181,300MAG006,IGE,2.0


In [186]:
temp_pivot = temp.pivot(index=['key_estudiante','periodo','asignatura'], columns='departamento', values='promedio').reset_index()
prefijo = 'prom_nivel_dpto_'
temp_pivot = temp_pivot.add_prefix(prefijo)
temp_pivot.rename(columns={prefijo + 'key_estudiante':'key_estudiante',
                           prefijo + 'periodo':'periodo',
                           prefijo + 'asignatura':'asignatura'}, 
                           inplace=True)
temp_pivot.head()

departamento,key_estudiante,periodo,asignatura,prom_nivel_dpto_31,prom_nivel_dpto_AAF,prom_nivel_dpto_ADM,prom_nivel_dpto_AGF,prom_nivel_dpto_AGG,prom_nivel_dpto_ANB,prom_nivel_dpto_ANC,...,prom_nivel_dpto_MED,prom_nivel_dpto_NTD,prom_nivel_dpto_OBA,prom_nivel_dpto_PSC,prom_nivel_dpto_PSG,prom_nivel_dpto_PSV,prom_nivel_dpto_RLD,prom_nivel_dpto_SAP,prom_nivel_dpto_TEG,prom_nivel_dpto_TMS
0,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20181,300MAG006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN
1,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20181,300MAG007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN
2,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20182,300MAG006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN
3,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20182,300MAG007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN
4,00724CD562AF1C6FF4ED73991F1DA0BB174A0727,20182,300MAG006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN


In [187]:
temp_pivot.shape

(7081, 123)

In [188]:
DATOS = DATOS.merge(temp_pivot, on=['key_estudiante','periodo','asignatura'], how='left')
DATOS = DATOS.fillna(-10) # -10: si no es posible calcular dpto cursos anteriores
DATOS.head()

,key_estudiante,periodo,asignatura,estudiante_edad,estudiante_genero,colegio,key_docente,docente_genero,docente_edad,estado,...,prom_nivel_dpto_MED,prom_nivel_dpto_NTD,prom_nivel_dpto_OBA,prom_nivel_dpto_PSC,prom_nivel_dpto_PSG,prom_nivel_dpto_PSV,prom_nivel_dpto_RLD,prom_nivel_dpto_SAP,prom_nivel_dpto_TEG,prom_nivel_dpto_TMS
0,306023AB9393647A8E4AD19E67EC6AB620240D78,20151,300MAG006,18,1,0047377,Ninguno,1,43,E,...,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0
1,306023AB9393647A8E4AD19E67EC6AB620240D78,20152,300MAG007,19,1,0047377,Ninguno,1,43,E,...,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-2.0,-10.0
2,1E8888869DC4F3ED9EC8F786AC7377C44CAAEA3D,20152,300MAG006,19,1,0203539,Ninguno,1,43,E,...,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0
3,1BF76E980C707F5F18F9C3FC6EC18D828DC371CA,20152,300MAG006,18,1,0206317,Ninguno,1,43,E,...,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0
4,1917B5D7ADFF357668C8A6F0A8EAF0638DF6B1E7,20152,300MAG006,17,1,0047257,Ninguno,1,43,E,...,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,1.0,-10.0


In [189]:
print(DATOS.shape)
DATOS.to_pickle('checkPoints/DATOS4.pkl')

(7145, 1813)


## **5. promedio nivel nota por asignaturas anteriores**

Promedio nivel_nota obtenido de semestres anteriores.
Se calcula el promedio de nivel nota por asignatra, se mira el orden de este y se calcula
la diferencia contra el orden del periodo actual.

Es el promedio pues pudo haberse visto una misma asigantura más de una vez

In [190]:
DATOS = pd.read_pickle('checkPoints/DATOS4.pkl')

# decido usar mysql-connect en vez de alchemy porque hacer un DML con alchemy es un dolor de huevo
query = """
    SELECT ma.key_estudiante, ma.periodo, ma.asignatura, 
           ma2.asignatura asig_ant, AVG(ma2.nivel_nota) promedio
    FROM f_matriculasAsignaturas ma						-- tabla base
    INNER JOIN f_periodos p								--  periodo de ma
    ON p.periodo = ma.periodo
    INNER JOIN  f_matriculasAsignaturas ma2				-- las asignaturas que vio antes
    ON  ma2.key_estudiante = ma.key_estudiante
    INNER JOIN f_periodos p2							-- periodo de las asignaturas que vio antes
    ON p2.periodo = ma2.periodo
    WHERE ma.analizar = 1								-- se toman en cuenta solo analizar = 1
    AND p2.orden < p.orden                          -- solo miramos en el orden correcto
    GROUP BY ma.key_estudiante, ma.periodo, ma.asignatura, ma2.asignatura;
"""
temp = pd.read_sql(query, con=motor)
temp.head()

,key_estudiante,periodo,asignatura,asig_ant,promedio
0,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20181,300MAG006,300CIP001,2.0
1,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20181,300MAG006,300CSP003,2.0
2,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20181,300MAG006,300EIH001,2.0
3,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20181,300MAG006,300EPG002,2.0
4,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20181,300MAG006,300IGE001,2.0


In [191]:
temp_pivot = temp.pivot(index=['key_estudiante','periodo','asignatura'], columns='asig_ant', values='promedio').reset_index()
prefijo = 'prom_nivel_asig_'
temp_pivot = temp_pivot.add_prefix(prefijo)
temp_pivot.rename(columns={prefijo + 'key_estudiante':'key_estudiante',
                           prefijo + 'periodo':'periodo',
                           prefijo + 'asignatura':'asignatura'}, 
                           inplace=True)
temp_pivot.head()

asig_ant,key_estudiante,periodo,asignatura,prom_nivel_asig_300AAF001,prom_nivel_asig_300AAF002,prom_nivel_asig_300AAF003,prom_nivel_asig_300AAF004,prom_nivel_asig_300AAF006,prom_nivel_asig_300AAF007,prom_nivel_asig_300AAF008,...,prom_nivel_asig_300PSV002,prom_nivel_asig_300RLD001,prom_nivel_asig_300SAP004,prom_nivel_asig_300TEG001,prom_nivel_asig_300TEG002,prom_nivel_asig_300TMS001,prom_nivel_asig_300TMS002,prom_nivel_asig_300TMS006,prom_nivel_asig_303IBA002,prom_nivel_asig_MT131
0,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20181,300MAG006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN
1,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20181,300MAG007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN
2,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20182,300MAG006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2.0,2.0,NaN,NaN,NaN,NaN,NaN
3,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20182,300MAG007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2.0,2.0,NaN,NaN,NaN,NaN,NaN
4,00724CD562AF1C6FF4ED73991F1DA0BB174A0727,20182,300MAG006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN


In [192]:
DATOS = DATOS.merge(temp_pivot, on=['key_estudiante','periodo','asignatura'], how='left')
DATOS = DATOS.fillna(-10) # -10: si no es posible calcular asig cursos anteriores
DATOS.head()

,key_estudiante,periodo,asignatura,estudiante_edad,estudiante_genero,colegio,key_docente,docente_genero,docente_edad,estado,...,prom_nivel_asig_300PSV002,prom_nivel_asig_300RLD001,prom_nivel_asig_300SAP004,prom_nivel_asig_300TEG001,prom_nivel_asig_300TEG002,prom_nivel_asig_300TMS001,prom_nivel_asig_300TMS002,prom_nivel_asig_300TMS006,prom_nivel_asig_303IBA002,prom_nivel_asig_MT131
0,306023AB9393647A8E4AD19E67EC6AB620240D78,20151,300MAG006,18,1,0047377,Ninguno,1,43,E,...,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0
1,306023AB9393647A8E4AD19E67EC6AB620240D78,20152,300MAG007,19,1,0047377,Ninguno,1,43,E,...,-10.0,-10.0,-10.0,1.0,-5.0,-10.0,-10.0,-10.0,-10.0,-10.0
2,1E8888869DC4F3ED9EC8F786AC7377C44CAAEA3D,20152,300MAG006,19,1,0203539,Ninguno,1,43,E,...,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0
3,1BF76E980C707F5F18F9C3FC6EC18D828DC371CA,20152,300MAG006,18,1,0206317,Ninguno,1,43,E,...,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0
4,1917B5D7ADFF357668C8A6F0A8EAF0638DF6B1E7,20152,300MAG006,17,1,0047257,Ninguno,1,43,E,...,-10.0,-10.0,-10.0,1.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0


In [193]:
print(DATOS.shape)
DATOS.to_pickle('checkPoints/DATOS5.pkl')

(7145, 2498)


## **6. promedio nivel nota por numero de creditos anteriores**

Promedio nivel_nota obtenido de semestres anteriores.
Se calcula el promedio de nivel nota por numero de creditos, 

Es el promedio pues pudo haberse visto una misma asigantura más de una vez

In [194]:
DATOS = pd.read_pickle('checkPoints/DATOS5.pkl')

# decido usar mysql-connect en vez de alchemy porque hacer un DML con alchemy es un dolor de huevo
query = """
    SELECT ma.key_estudiante, ma.periodo, ma.asignatura, 
           ma2.creditos, AVG(ma2.nivel_nota) promedio
    FROM f_matriculasAsignaturas ma						-- tabla base
    INNER JOIN f_periodos p								--  periodo de ma
    ON p.periodo = ma.periodo
    INNER JOIN  f_matriculasAsignaturas ma2				-- las asignaturas que vio antes
    ON  ma2.key_estudiante = ma.key_estudiante
    INNER JOIN f_periodos p2							-- periodo de las asignaturas que vio antes
    ON p2.periodo = ma2.periodo
    WHERE ma.analizar = 1								-- se toman en cuenta solo analizar = 1
    AND p2.orden < p.orden                          -- solo miramos en el orden correcto, tanto CV como RE
    GROUP BY ma.key_estudiante, ma.periodo, ma.asignatura, ma2.creditos;
"""
temp = pd.read_sql(query, con=motor)
temp.head()

,key_estudiante,periodo,asignatura,creditos,promedio
0,007AD6584DA00DEE96BA57A369CDAB8C9C832D53,20182,300MAG007,3,1.7778
1,007AD6584DA00DEE96BA57A369CDAB8C9C832D53,20182,300MAG007,2,1.8333
2,AA8834B56776F3D46C0C7983349F9515B1A686E7,20181,300MAG007,3,0.8462
3,AA8834B56776F3D46C0C7983349F9515B1A686E7,20181,300MAG007,4,1.0000
4,AA8834B56776F3D46C0C7983349F9515B1A686E7,20181,300MAG007,2,1.0909


In [195]:
temp_pivot = temp.pivot(index=['key_estudiante','periodo','asignatura'], columns='creditos', values='promedio').reset_index()
prefijo = 'prom_nivel_creditos_'
temp_pivot = temp_pivot.add_prefix(prefijo)
temp_pivot.rename(columns={prefijo + 'key_estudiante':'key_estudiante',
                           prefijo + 'periodo':'periodo',
                           prefijo + 'asignatura':'asignatura'}, 
                           inplace=True)
temp_pivot.head()

creditos,key_estudiante,periodo,asignatura,prom_nivel_creditos_0,prom_nivel_creditos_1,prom_nivel_creditos_2,prom_nivel_creditos_3,prom_nivel_creditos_4,prom_nivel_creditos_5,prom_nivel_creditos_6,prom_nivel_creditos_7,prom_nivel_creditos_8,prom_nivel_creditos_9,prom_nivel_creditos_12,prom_nivel_creditos_15
0,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20181,300MAG006,NaN,2.0,2.0000,2.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20181,300MAG007,NaN,2.0,2.0000,2.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20182,300MAG006,NaN,2.0,2.0000,2.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20182,300MAG007,NaN,2.0,2.0000,2.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,00724CD562AF1C6FF4ED73991F1DA0BB174A0727,20182,300MAG006,NaN,NaN,1.1667,-0.125,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [196]:
DATOS = DATOS.merge(temp_pivot, on=['key_estudiante','periodo','asignatura'], how='left')
DATOS = DATOS.fillna(-10) # -10: si no es posible calcular asig cursos anteriores
DATOS.head()

,key_estudiante,periodo,asignatura,estudiante_edad,estudiante_genero,colegio,key_docente,docente_genero,docente_edad,estado,...,prom_nivel_creditos_2,prom_nivel_creditos_3,prom_nivel_creditos_4,prom_nivel_creditos_5,prom_nivel_creditos_6,prom_nivel_creditos_7,prom_nivel_creditos_8,prom_nivel_creditos_9,prom_nivel_creditos_12,prom_nivel_creditos_15
0,306023AB9393647A8E4AD19E67EC6AB620240D78,20151,300MAG006,18,1,0047377,Ninguno,1,43,E,...,1.4000,1.0000,1.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0
1,306023AB9393647A8E4AD19E67EC6AB620240D78,20152,300MAG007,19,1,0047377,Ninguno,1,43,E,...,0.5000,0.9091,1.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0
2,1E8888869DC4F3ED9EC8F786AC7377C44CAAEA3D,20152,300MAG006,19,1,0203539,Ninguno,1,43,E,...,1.3333,1.0000,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0
3,1BF76E980C707F5F18F9C3FC6EC18D828DC371CA,20152,300MAG006,18,1,0206317,Ninguno,1,43,E,...,1.0000,0.6667,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0
4,1917B5D7ADFF357668C8A6F0A8EAF0638DF6B1E7,20152,300MAG006,17,1,0047257,Ninguno,1,43,E,...,1.2500,1.5000,1.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0


In [197]:
DATOS.to_pickle('checkPoints/DATOS6.pkl')
print(DATOS.shape)

(7145, 2510)


## **7. cantidad de cursos que ya había tomado el estudiante antes de tomar el curso actual**

se calcula el total de cursos que ya había tomado el estudiante antes de tomar este (sin contar los matriculados este semestre).  SE incluyen cursos tomados en CV y RE, y si repitió un curso se cuenta 2 veces o más

In [198]:
DATOS = pd.read_pickle('checkPoints/DATOS6.pkl')

# decido usar mysql-connect en vez de alchemy porque hacer un DML con alchemy es un dolor de huevo
query = """
    SELECT ma.key_estudiante, ma.periodo, ma.asignatura, count(*) q_cursos_tomados
    FROM f_matriculasAsignaturas ma						-- asignaturas a tomar en cuenta
    INNER JOIN f_periodos p								--  periodo de ma
    ON p.periodo = ma.periodo
    INNER JOIN  f_matriculasAsignaturas ma2				-- las asignaturas que vio antes
    ON  ma2.key_estudiante = ma.key_estudiante
    INNER JOIN f_periodos p2							-- periodo de las asignaturas que vio antes
    ON p2.periodo = ma2.periodo
    WHERE ma.analizar = 1								-- se toman en cuenta solo analizar = 1
    AND p2.orden < p.orden     						-- solo se miran periodos anteriores
    GROUP BY ma.key_estudiante, ma.periodo, ma.asignatura;
"""
temp = pd.read_sql(query, con=motor)
temp.head()

,key_estudiante,periodo,asignatura,q_cursos_tomados
0,007AD6584DA00DEE96BA57A369CDAB8C9C832D53,20182,300MAG007,15
1,AA8834B56776F3D46C0C7983349F9515B1A686E7,20181,300MAG007,27
2,B6528A285FD0E44402DD52429EDC8BAA093429F5,20181,300MAG006,26
3,EF8F28CA573273EF9B64853FF1A3D931BBAB9D73,20181,300MAG007,29
4,6C9E49411AD94E2B16A9D0EAD160C15C7F3D9735,20182,300MAG006,23


In [199]:
DATOS = DATOS.merge(temp, on=['key_estudiante','periodo','asignatura'], how='left')
DATOS = DATOS.fillna(0) # 0: si no tomado cursos anteriores
DATOS.head()

,key_estudiante,periodo,asignatura,estudiante_edad,estudiante_genero,colegio,key_docente,docente_genero,docente_edad,estado,...,prom_nivel_creditos_3,prom_nivel_creditos_4,prom_nivel_creditos_5,prom_nivel_creditos_6,prom_nivel_creditos_7,prom_nivel_creditos_8,prom_nivel_creditos_9,prom_nivel_creditos_12,prom_nivel_creditos_15,q_cursos_tomados
0,306023AB9393647A8E4AD19E67EC6AB620240D78,20151,300MAG006,18,1,0047377,Ninguno,1,43,E,...,1.0000,1.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,13.0
1,306023AB9393647A8E4AD19E67EC6AB620240D78,20152,300MAG007,19,1,0047377,Ninguno,1,43,E,...,0.9091,1.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,21.0
2,1E8888869DC4F3ED9EC8F786AC7377C44CAAEA3D,20152,300MAG006,19,1,0203539,Ninguno,1,43,E,...,1.0000,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,7.0
3,1BF76E980C707F5F18F9C3FC6EC18D828DC371CA,20152,300MAG006,18,1,0206317,Ninguno,1,43,E,...,0.6667,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,7.0
4,1917B5D7ADFF357668C8A6F0A8EAF0638DF6B1E7,20152,300MAG006,17,1,0047257,Ninguno,1,43,E,...,1.5000,1.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,7.0


In [200]:
DATOS.to_pickle('checkPoints/DATOS7.pkl')
print(DATOS.shape)

(7145, 2511)


## **8. cantidad de semestres regulares que ya lleva el estudiante en la U al tomar el curso**

se calcula el total periodos regulares que lleva el estudiante en la U antes de tomar el curso

In [201]:
DATOS = pd.read_pickle('checkPoints/DATOS7.pkl')

# decido usar mysql-connect en vez de alchemy porque hacer un DML con alchemy es un dolor de huevo
query = """
    SELECT key_estudiante, periodo, asignatura, count(*) q_semestres_reg_tomados
    FROM (
        SELECT DISTINCT ma.key_estudiante, ma.periodo, ma.asignatura, ma2.periodo periodo_ant
        FROM f_matriculasAsignaturas ma						-- asignaturas a tomar en cuenta
        INNER JOIN f_periodos p								--  periodo de ma
        ON p.periodo = ma.periodo
        INNER JOIN  f_matriculasAsignaturas ma2				-- las asignaturas que vio antes
        ON  ma2.key_estudiante = ma.key_estudiante
        INNER JOIN f_periodos p2							-- periodo de las asignaturas que vio antes
        ON p2.periodo = ma2.periodo
        WHERE ma.analizar = 1								-- se toman en cuenta solo analizar = 1
        AND p2.orden < p.orden     						-- solo se miran periodos anteriores
    ) V
    GROUP BY key_estudiante, periodo, asignatura;
"""
temp = pd.read_sql(query, con=motor)
temp.head()

,key_estudiante,periodo,asignatura,q_semestres_reg_tomados
0,007AD6584DA00DEE96BA57A369CDAB8C9C832D53,20182,300MAG007,2
1,AA8834B56776F3D46C0C7983349F9515B1A686E7,20181,300MAG007,4
2,B6528A285FD0E44402DD52429EDC8BAA093429F5,20181,300MAG006,4
3,EF8F28CA573273EF9B64853FF1A3D931BBAB9D73,20181,300MAG007,5
4,6C9E49411AD94E2B16A9D0EAD160C15C7F3D9735,20182,300MAG006,3


In [202]:
DATOS = DATOS.merge(temp, on=['key_estudiante','periodo','asignatura'], how='left')
DATOS = DATOS.fillna(0) # 0: si no tomado cursos anteriores
DATOS.head()

,key_estudiante,periodo,asignatura,estudiante_edad,estudiante_genero,colegio,key_docente,docente_genero,docente_edad,estado,...,prom_nivel_creditos_4,prom_nivel_creditos_5,prom_nivel_creditos_6,prom_nivel_creditos_7,prom_nivel_creditos_8,prom_nivel_creditos_9,prom_nivel_creditos_12,prom_nivel_creditos_15,q_cursos_tomados,q_semestres_reg_tomados
0,306023AB9393647A8E4AD19E67EC6AB620240D78,20151,300MAG006,18,1,0047377,Ninguno,1,43,E,...,1.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,13.0,2.0
1,306023AB9393647A8E4AD19E67EC6AB620240D78,20152,300MAG007,19,1,0047377,Ninguno,1,43,E,...,1.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,21.0,4.0
2,1E8888869DC4F3ED9EC8F786AC7377C44CAAEA3D,20152,300MAG006,19,1,0203539,Ninguno,1,43,E,...,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,7.0,1.0
3,1BF76E980C707F5F18F9C3FC6EC18D828DC371CA,20152,300MAG006,18,1,0206317,Ninguno,1,43,E,...,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,7.0,1.0
4,1917B5D7ADFF357668C8A6F0A8EAF0638DF6B1E7,20152,300MAG006,17,1,0047257,Ninguno,1,43,E,...,1.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,7.0,1.0


In [203]:
DATOS.to_pickle('checkPoints/DATOS8.pkl')
print(DATOS.shape)

(7145, 2512)


## **9. cantidad de cursos que tomó en cursos de verano**

se calcula de cursos que tomó en períodos CV anteriore al curso actual

In [204]:
DATOS = pd.read_pickle('checkPoints/DATOS8.pkl')

query = """
    SELECT ma.key_estudiante, ma.periodo, ma.asignatura, count(*) q_cursos_CV
    FROM f_matriculasAsignaturas ma
    INNER JOIN f_periodos p
    ON p.periodo = ma.periodo
    INNER JOIN f_matriculasAsignaturas ma2
    ON ma2.key_estudiante = ma.key_estudiante
    INNER JOIN  f_periodos p2
    ON p2.periodo = ma2.periodo
    WHERE ma.analizar = 1						-- tomamos en cuenta solo las asignaturas de interés
    AND p2.tipo ='CV'							-- los periodos anteriores solo se toman en cuenta los tipo CV
    AND p2.orden < p.orden						-- solo tomamos en cuenta periodos anteriores al periodo de la asig
    GROUP BY ma.key_estudiante, ma.periodo, ma.asignatura;
    """
temp = pd.read_sql(query, con=motor)
temp.head()

,key_estudiante,periodo,asignatura,q_cursos_CV
0,EF8F28CA573273EF9B64853FF1A3D931BBAB9D73,20181,300MAG007,2
1,C3721A911631A071D1254123E4FDEAB997F85313,20202,300MAG007,1
2,5016C395537ADAEB7319B385CEB3E3E1DA8CB3CB,20181,300MAG007,2
3,10A44812801FDBB3C975526B1715EE1A86737E38,20181,300MAG006,1
4,5A5FC6C246E652812D05702199FE36688EEC98E8,20181,300MAG007,1


In [205]:
DATOS = DATOS.merge(temp, on=['key_estudiante','periodo','asignatura'], how='left')
DATOS = DATOS.fillna(0) # 0: si no tomado cursos anteriores tipo CV
DATOS.head()

,key_estudiante,periodo,asignatura,estudiante_edad,estudiante_genero,colegio,key_docente,docente_genero,docente_edad,estado,...,prom_nivel_creditos_5,prom_nivel_creditos_6,prom_nivel_creditos_7,prom_nivel_creditos_8,prom_nivel_creditos_9,prom_nivel_creditos_12,prom_nivel_creditos_15,q_cursos_tomados,q_semestres_reg_tomados,q_cursos_CV
0,306023AB9393647A8E4AD19E67EC6AB620240D78,20151,300MAG006,18,1,0047377,Ninguno,1,43,E,...,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,13.0,2.0,0.0
1,306023AB9393647A8E4AD19E67EC6AB620240D78,20152,300MAG007,19,1,0047377,Ninguno,1,43,E,...,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,21.0,4.0,1.0
2,1E8888869DC4F3ED9EC8F786AC7377C44CAAEA3D,20152,300MAG006,19,1,0203539,Ninguno,1,43,E,...,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,7.0,1.0,0.0
3,1BF76E980C707F5F18F9C3FC6EC18D828DC371CA,20152,300MAG006,18,1,0206317,Ninguno,1,43,E,...,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,7.0,1.0,0.0
4,1917B5D7ADFF357668C8A6F0A8EAF0638DF6B1E7,20152,300MAG006,17,1,0047257,Ninguno,1,43,E,...,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,7.0,1.0,0.0


In [206]:
DATOS.to_pickle('checkPoints/DATOS9.pkl')
print(DATOS.shape)

(7145, 2513)


## **10. cantidad de cursos que tomó en cursos de verano por cpto**

se calcula de cursos que tomó en períodos CV anteriore al curso actual, por dpto

In [207]:
DATOS = pd.read_pickle('checkPoints/DATOS9.pkl')

query = """
    SELECT ma.key_estudiante, ma.periodo, ma.asignatura, ma2.departamento, count(*) q_cursos_CV
    FROM f_matriculasAsignaturas ma
    INNER JOIN f_periodos p
    ON p.periodo = ma.periodo
    INNER JOIN f_matriculasAsignaturas ma2
    ON ma2.key_estudiante = ma.key_estudiante
    INNER JOIN  f_periodos p2
    ON p2.periodo = ma2.periodo
    WHERE ma.analizar = 1						-- tomamos en cuenta solo las asignaturas de interés
    AND p2.tipo ='CV'							-- los periodos anteriores solo se toman en cuenta los tipo CV
    AND p2.orden < p.orden						-- solo tomamos en cuenta periodos anteriores al periodo de la asig
    GROUP BY ma.key_estudiante, ma.periodo, ma.asignatura, ma2.departamento;
    """
temp = pd.read_sql(query, con=motor)
temp.head()

,key_estudiante,periodo,asignatura,departamento,q_cursos_CV
0,EF8F28CA573273EF9B64853FF1A3D931BBAB9D73,20181,300MAG007,MAG,2
1,C3721A911631A071D1254123E4FDEAB997F85313,20202,300MAG007,MAG,1
2,5016C395537ADAEB7319B385CEB3E3E1DA8CB3CB,20181,300MAG007,MAG,2
3,10A44812801FDBB3C975526B1715EE1A86737E38,20181,300MAG006,MAG,1
4,5A5FC6C246E652812D05702199FE36688EEC98E8,20181,300MAG007,MAG,1


In [208]:
temp_pivot = temp.pivot(index=['key_estudiante','periodo','asignatura'], columns='departamento', values='q_cursos_CV').reset_index()
prefijo = 'q_cursos_CV_dpto_'
temp_pivot = temp_pivot.add_prefix(prefijo)
temp_pivot.rename(columns={prefijo + 'key_estudiante':'key_estudiante',
                           prefijo + 'periodo':'periodo',
                           prefijo + 'asignatura':'asignatura'}, 
                           inplace=True)
temp_pivot.head()

departamento,key_estudiante,periodo,asignatura,q_cursos_CV_dpto_AQA,q_cursos_CV_dpto_LEI,q_cursos_CV_dpto_MAE,q_cursos_CV_dpto_MAG,q_cursos_CV_dpto_MDM
0,00844F609DA54DFF214529B66202E11E4D020184,20201,300MAG006,NaN,NaN,NaN,1.0,NaN
1,00844F609DA54DFF214529B66202E11E4D020184,20201,300MAG007,NaN,NaN,NaN,1.0,NaN
2,00844F609DA54DFF214529B66202E11E4D020184,20211,300MAG006,NaN,NaN,NaN,1.0,NaN
3,00844F609DA54DFF214529B66202E11E4D020184,20211,300MAG007,NaN,NaN,NaN,1.0,NaN
4,0163D087A12913268CBEB4C483BE1F6B6A4EB7F2,20222,300MAG006,NaN,NaN,NaN,1.0,NaN


In [209]:
DATOS = DATOS.merge(temp_pivot, on=['key_estudiante','periodo','asignatura'], how='left')
DATOS = DATOS.fillna(0) # 0: si no tomado cursos anteriores tipo CV
DATOS.head()

,key_estudiante,periodo,asignatura,estudiante_edad,estudiante_genero,colegio,key_docente,docente_genero,docente_edad,estado,...,prom_nivel_creditos_12,prom_nivel_creditos_15,q_cursos_tomados,q_semestres_reg_tomados,q_cursos_CV,q_cursos_CV_dpto_AQA,q_cursos_CV_dpto_LEI,q_cursos_CV_dpto_MAE,q_cursos_CV_dpto_MAG,q_cursos_CV_dpto_MDM
0,306023AB9393647A8E4AD19E67EC6AB620240D78,20151,300MAG006,18,1,0047377,Ninguno,1,43,E,...,-10.0,-10.0,13.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
1,306023AB9393647A8E4AD19E67EC6AB620240D78,20152,300MAG007,19,1,0047377,Ninguno,1,43,E,...,-10.0,-10.0,21.0,4.0,1.0,0.0,0.0,0.0,1.0,0.0
2,1E8888869DC4F3ED9EC8F786AC7377C44CAAEA3D,20152,300MAG006,19,1,0203539,Ninguno,1,43,E,...,-10.0,-10.0,7.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1BF76E980C707F5F18F9C3FC6EC18D828DC371CA,20152,300MAG006,18,1,0206317,Ninguno,1,43,E,...,-10.0,-10.0,7.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1917B5D7ADFF357668C8A6F0A8EAF0638DF6B1E7,20152,300MAG006,17,1,0047257,Ninguno,1,43,E,...,-10.0,-10.0,7.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [210]:
DATOS.to_pickle('checkPoints/DATOS10.pkl')
print(DATOS.shape)

(7145, 2518)


## **11. cuales cursos  tomó en cursos de verano por profe**

lista de cursos tomados en curso de verano por profe.  el valor es la cantidad (se puede repetir asig profe más de una vez)

In [215]:
DATOS = pd.read_pickle('checkPoints/DATOS10.pkl')

query = """
    SELECT ma.key_estudiante, ma.periodo, ma.asignatura,  d.id_docente, count(*) cuantos
    FROM f_matriculasAsignaturas ma
    INNER JOIN f_periodos p
    ON p.periodo = ma.periodo
    INNER JOIN f_matriculasAsignaturas ma2
    ON ma2.key_estudiante = ma.key_estudiante
    INNER JOIN  f_periodos p2
    ON p2.periodo = ma2.periodo
    INNER JOIN f_docentes d
    ON d.key_docente = ma2.key_docente
    WHERE ma.analizar = 1						-- tomamos en cuenta solo las asignaturas de interés
    AND p2.tipo ='CV'							-- los periodos anteriores solo se toman en cuenta los tipo CV
    AND p2.orden < p.orden						-- solo tomamos en cuenta periodos anteriores al periodo de la asig
    GROUP BY ma.key_estudiante, ma.periodo, ma.asignatura, d.id_docente;
    """
temp = pd.read_sql(query, con=motor)
temp.head()

,key_estudiante,periodo,asignatura,id_docente,cuantos
0,EEE57B49600472B19A8A1D874D2E2C5ADCFB5EB2,20201,300MAG006,22,1
1,59D674FDA540D1EC44215D55FFE203406C1D8FC9,20201,300MAG007,20,1
2,59D674FDA540D1EC44215D55FFE203406C1D8FC9,20201,300MAG006,20,1
3,EEE57B49600472B19A8A1D874D2E2C5ADCFB5EB2,20201,300MAG007,22,1
4,DA27C2E31809E63D56F3B69A18F10050CF5C6C87,20182,300MAG006,16,1


In [217]:
temp_pivot = temp.pivot(index=['key_estudiante','periodo','asignatura'], columns='id_docente', values='cuantos').reset_index()
prefijo = 'q_cursos_CV_docente_'
temp_pivot = temp_pivot.add_prefix(prefijo)
temp_pivot.rename(columns={prefijo + 'key_estudiante':'key_estudiante',
                           prefijo + 'periodo':'periodo',
                           prefijo + 'asignatura':'asignatura'}, 
                           inplace=True)
temp_pivot.head()

id_docente,key_estudiante,periodo,asignatura,q_cursos_CV_docente_14,q_cursos_CV_docente_16,q_cursos_CV_docente_20,q_cursos_CV_docente_22,q_cursos_CV_docente_24,q_cursos_CV_docente_29
0,03E1E4836166AC1FD3C2F0E415F3578D91656E35,20231,300MAG006,NaN,NaN,NaN,NaN,NaN,2.0
1,03E1E4836166AC1FD3C2F0E415F3578D91656E35,20231,300MAG007,NaN,NaN,NaN,NaN,NaN,2.0
2,0642AA8AE9E5CA66990F2427D0CDA2477972BB2C,20191,300MAG006,NaN,NaN,1.0,NaN,NaN,NaN
3,0642AA8AE9E5CA66990F2427D0CDA2477972BB2C,20191,300MAG007,NaN,NaN,1.0,NaN,NaN,NaN
4,2338F23C514CFEBC29C100981B55DB7B41624778,20191,300MAG006,NaN,NaN,1.0,NaN,NaN,NaN


In [218]:
DATOS = DATOS.merge(temp_pivot, on=['key_estudiante','periodo','asignatura'], how='left')
DATOS = DATOS.fillna(0) # 0: si no tomado cursos anteriores tipo CV
DATOS.to_pickle('checkPoints/DATOS11.pkl')
print(DATOS.shape)
DATOS.head()

(7145, 2524)


,key_estudiante,periodo,asignatura,estudiante_edad,estudiante_genero,colegio,key_docente,docente_genero,docente_edad,estado,...,q_cursos_CV_dpto_LEI,q_cursos_CV_dpto_MAE,q_cursos_CV_dpto_MAG,q_cursos_CV_dpto_MDM,q_cursos_CV_docente_14,q_cursos_CV_docente_16,q_cursos_CV_docente_20,q_cursos_CV_docente_22,q_cursos_CV_docente_24,q_cursos_CV_docente_29
0,306023AB9393647A8E4AD19E67EC6AB620240D78,20151,300MAG006,18,1,0047377,Ninguno,1,43,E,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,306023AB9393647A8E4AD19E67EC6AB620240D78,20152,300MAG007,19,1,0047377,Ninguno,1,43,E,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1E8888869DC4F3ED9EC8F786AC7377C44CAAEA3D,20152,300MAG006,19,1,0203539,Ninguno,1,43,E,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1BF76E980C707F5F18F9C3FC6EC18D828DC371CA,20152,300MAG006,18,1,0206317,Ninguno,1,43,E,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1917B5D7ADFF357668C8A6F0A8EAF0638DF6B1E7,20152,300MAG006,17,1,0047257,Ninguno,1,43,E,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## **12. promedio semestre RE hace**

Promedio de semestres regulares anteriores.   
OJO hay que tener cuidado.   Si solo quiero calcular distancias contra periodos regulares, hay que entender que la fila f_matriculasAsignaturas que se analiza puede ser RE o CV.  los CV no tienen ordenRE y por tanto no puede calcularse la distancia.

Si se quiere calcular distancias contra RE, colocar la condicion de RE para periodos de f_matriculasAsignaturas, pero usar orden y no ordenRE pues si la fila de f_matriculasAsignatura es CV no va a tener orderRE (será NULO) y no podrá calcularse las distancia sy seguro saldrna filas repetidas.



In [11]:
DATOS = pd.read_pickle('checkPoints/DATOS11.pkl')

query = """
    SELECT  ma.key_estudiante, ma.periodo, ma.asignatura,
            p.orden - p2.orden hace, m.promedio_semestre
    FROM f_matriculasAsignaturas ma
    INNER JOIN f_periodos p
    ON p.periodo = ma.periodo
    INNER JOIN f_matriculas m
    ON m.key_estudiante = ma.key_estudiante
    INNER JOIN  f_periodos p2
    ON p2.periodo = m.periodo
    WHERE ma.analizar = 1						-- tomamos en cuenta solo las asignaturas de interés
    AND p2.orden < p.orden;
"""
temp = pd.read_sql(query, con=motor)
temp.head()

,key_estudiante,periodo,asignatura,hace,promedio_semestre
0,306023AB9393647A8E4AD19E67EC6AB620240D78,20151,300MAG006,1,3.64
1,306023AB9393647A8E4AD19E67EC6AB620240D78,20152,300MAG007,3,3.64
2,306023AB9393647A8E4AD19E67EC6AB620240D78,20161,300MAG007,4,3.64
3,306023AB9393647A8E4AD19E67EC6AB620240D78,20162,300MAG007,6,3.64
4,306023AB9393647A8E4AD19E67EC6AB620240D78,20171,300MAG007,8,3.64


In [13]:
temp_pivot = temp.pivot(index=['key_estudiante','periodo','asignatura'], 
                        columns='hace', values='promedio_semestre').reset_index()
prefijo = 'promedio_sem_hace_'
temp_pivot = temp_pivot.add_prefix(prefijo)
temp_pivot.rename(columns={prefijo + 'key_estudiante':'key_estudiante',
                           prefijo + 'periodo':'periodo',
                           prefijo + 'asignatura':'asignatura'}, 
                           inplace=True)
temp_pivot.head()

hace,key_estudiante,periodo,asignatura,promedio_sem_hace_1,promedio_sem_hace_2,promedio_sem_hace_3,promedio_sem_hace_4,promedio_sem_hace_5,promedio_sem_hace_6,promedio_sem_hace_7,...,promedio_sem_hace_24,promedio_sem_hace_25,promedio_sem_hace_27,promedio_sem_hace_28,promedio_sem_hace_29,promedio_sem_hace_30,promedio_sem_hace_32,promedio_sem_hace_33,promedio_sem_hace_34,promedio_sem_hace_35
0,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20181,300MAG006,NaN,4.78,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20181,300MAG007,NaN,4.78,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20182,300MAG006,NaN,4.79,NaN,4.78,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,002F74930C7956A25CCEEECF8CEF84C6FC41C66C,20182,300MAG007,NaN,4.79,NaN,4.78,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,00724CD562AF1C6FF4ED73991F1DA0BB174A0727,20182,300MAG006,NaN,2.23,NaN,2.94,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
DATOS = DATOS.merge(temp_pivot, on=['key_estudiante','periodo','asignatura'], how='left')
DATOS = DATOS.fillna(-10) # -10: si no existe la distancia hasta un RE (ej: el primer semestre)
DATOS.to_pickle('checkPoints/DATOS12.pkl')
print(DATOS.shape)
DATOS.head()

(7145, 2557)


,key_estudiante,periodo,asignatura,estudiante_edad,estudiante_genero,colegio,key_docente,docente_genero,docente_edad,estado,...,promedio_sem_hace_24,promedio_sem_hace_25,promedio_sem_hace_27,promedio_sem_hace_28,promedio_sem_hace_29,promedio_sem_hace_30,promedio_sem_hace_32,promedio_sem_hace_33,promedio_sem_hace_34,promedio_sem_hace_35
0,306023AB9393647A8E4AD19E67EC6AB620240D78,20151,300MAG006,18,1,0047377,Ninguno,1,43,E,...,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0
1,306023AB9393647A8E4AD19E67EC6AB620240D78,20152,300MAG007,19,1,0047377,Ninguno,1,43,E,...,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0
2,1E8888869DC4F3ED9EC8F786AC7377C44CAAEA3D,20152,300MAG006,19,1,0203539,Ninguno,1,43,E,...,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0
3,1BF76E980C707F5F18F9C3FC6EC18D828DC371CA,20152,300MAG006,18,1,0206317,Ninguno,1,43,E,...,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0
4,1917B5D7ADFF357668C8A6F0A8EAF0638DF6B1E7,20152,300MAG006,17,1,0047257,Ninguno,1,43,E,...,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0,-10.0


## **12. promedio semestre RE hace**

Promedio de semestres regulares anteriores.   
OJO hay que tener cuidado.   Si solo quiero calcular distancias contra periodos regulares, hay que entender que la fila f_matriculasAsignaturas que se analiza puede ser RE o CV.  los CV no tienen ordenRE y por tanto no puede calcularse la distancia.

Si se quiere calcular distancias contra RE, colocar la condicion de RE para periodos de f_matriculasAsignaturas, pero usar orden y no ordenRE pues si la fila de f_matriculasAsignatura es CV no va a tener orderRE (será NULO) y no podrá calcularse las distancia sy seguro saldrna filas repetidas.